In [ ]:

%matplotlib inline


In [ ]:
import sys
sys.path.insert(1, '/home/berkeleylab/Misc_code/HpBandSter/hpbandster')

In [ ]:


import core.nameserver as hpns
import core.result as hpres
from optimizers import BOHB as BOHB
from BeamWorker import BeamWorker
import visualization as hpvis
import matplotlib.pyplot as plt



In [ ]:

NS = hpns.NameServer(run_id='Beamville', host='127.0.0.1', port=None)
NS.start()


In [ ]:

workers=[]
for i in range(3):
   
    w = BeamWorker(nameserver='127.0.0.1', run_id='Beamville', id=i)
    w.run(background=True)
    workers.append(w)
 
    

In [ ]:

result_logger = hpres.json_result_logger(directory='/home/berkeleylab/Calib_documentation/TUNE_integrate/TUNE_test/worker_n_optimizer', overwrite=True)
bohb = BOHB(configspace=w.get_configspace(),run_id='Beamville',min_budget=1, max_budget=3, result_logger=result_logger)
res = bohb.run(n_iterations=3,min_n_workers=3)



In [ ]:

id2config = res.get_id2config_mapping()
incumbent = res.get_incumbent_id()
all_runs = res.get_all_runs()

print('best config:', id2config[incumbent]['config'] )
print('total unique configs:', len(id2config.keys()) )
print('total executed runs:', len(all_runs) )
print('total budget corresponds to these function evaluations:', (sum([r.budget for r in all_runs])/243))
print('total elapsed time', (all_runs[-1].time_stamps['finished']-all_runs[0].time_stamps['started']))



In [ ]:
bohb.shutdown(shutdown_workers=True)
NS.shutdown()


In [ ]:

result = hpres.logged_results_to_HBS_result('/home/berkeleylab/Calib_documentation/TUNE_integrate/TUNE_test/worker_n_optimizer/')


In [ ]:

all_runs = result.get_all_runs()
id2conf = result.get_id2config_mapping()
inc_id = result.get_incumbent_id()
inc_runs = result.get_runs_by_id(inc_id)
inc_run = inc_runs[-1]
inc_loss = inc_run.loss
inc_config = id2conf[inc_id]['config']
lcs = result.get_learning_curves()

print('Best config:', inc_loss)

In [ ]:
inc_runs

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
hpvis.losses_over_time(all_runs)

In [ ]:
hpvis.concurrent_runs_over_time(all_runs)

In [ ]:
hpvis.finished_runs_over_time(all_runs)

In [ ]:
hpvis.correlation_across_budgets(result)

In [ ]:
hpvis.performance_histogram_model_vs_random(all_runs,id2conf)

In [ ]:
def realtime_learning_curves(runs):
    sr = sorted(runs,key=lambda r: r.budget)
    lc = list(filter(lambda t: not t[1] is None, [(r.time_stamps['finished'], r.info) for r in sr]))
    
    return ([lc,])   
    
    

In [ ]:

lcs = result.get_learning_curves(lc_extractor=realtime_learning_curves)



In [ ]:
config_ids = [] 
times = []

for k in lcs.keys():
    config_ids.append(k)
    
for k,v in lcs.items():
    for l in v:
        if len(l) == 0: continue
        tmp = list(zip(*l))
        times.append(tmp[0])


In [ ]:

HB_iterations = [id[0] for id in config_ids]
num_iterations = len(set(HB_iterations))

HB_iterations

In [ ]:
num_iterations